In [ ]:
import ast
import os
import sys
import subprocess

# -------------------------
# Configuration
# -------------------------
project_folder = "../"  # parent folder
exclude_folders = {".venv", "__pycache__", ".ipynb_checkpoints", "data"}
pypi_overrides = {
    "googleapiclient": "google-api-python-client",
    "azure": "azure-functions",
}

# -------------------------
# Gather all .py files, skipping excluded folders
# -------------------------
python_files = []
for root, dirs, files in os.walk(project_folder):
    dirs[:] = [d for d in dirs if d not in exclude_folders]  # skip unwanted folders
    for f in files:
        if f.endswith(".py"):
            python_files.append(os.path.join(root, f))

if not python_files:
    print("No Python files found to scan. Check project_folder path.")
else:
    print(f"Found {len(python_files)} Python files to scan:")
    for f in python_files:
        print("-", f)

# -------------------------
# Parse imports from Python files
# -------------------------
imports = set()
for file_path in python_files:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            tree = ast.parse(f.read())
        for node in ast.walk(tree):
            if isinstance(node, ast.Import):
                for n in node.names:
                    imports.add(n.name.split(".")[0])
            elif isinstance(node, ast.ImportFrom):
                if node.module:
                    imports.add(node.module.split(".")[0])
    except UnicodeDecodeError:
        print("Skipping non-UTF-8 file:", file_path)

# -------------------------
# Remove standard library modules
# -------------------------
std_libs = set(sys.stdlib_module_names)
external_imports = [pypi_overrides.get(pkg, pkg) for pkg in imports if pkg not in std_libs]

if not external_imports:
    print("No external packages detected. Your code may use only standard library.")
else:
    print("Detected top-level external imports:")
    print(external_imports)

# -------------------------
# Optional: pin versions from current .venv
# -------------------------
requirements = []
for pkg in external_imports:
    try:
        version_info = subprocess.check_output(
            [sys.executable, "-m", "pip", "show", pkg],
            text=True
        )
        version = next((line.split(":")[1].strip() for line in version_info.splitlines() if line.startswith("Version:")), None)
        if version:
            requirements.append(f"{pkg}=={version}")
        else:
            requirements.append(pkg)
    except subprocess.CalledProcessError:
        requirements.append(pkg)  # package not installed in venv

# -------------------------
# Write minimal requirements.txt at project root
# -------------------------
requirements_path = os.path.join(project_folder, "requirements.txt")

with open(requirements_path, "w") as f:
    for r in sorted(requirements):
        f.write(r + "\n")

print(f"\nMinimal requirements.txt generated successfully at: {requirements_path}")
for r in requirements:
    print("-", r)


Found 3 Python files to scan:
- ../function_app.py
- ../test_scripts_folder/test_sql_server_connection.py
- ../test_scripts_folder/test_IP.py
Detected top-level external imports:
['pyodbc', 'requests', 'pytz', 'azure-functions']

Minimal requirements.txt generated successfully at: ../requirements.txt
- pyodbc==5.3.0
- requests==2.32.5
- pytz==2025.2
- azure-functions==1.24.0
